# **Avalanche Risk Project**

Advanced Data Analytics, Fall 2025

The following project examines the feasibility of machine learning models to predict dry avalanche danger from spatial and meteorological features. In a second step, the model will be trained on the whole of Switzerland and tested as well. 

In [25]:
# Necessary Libraries 
import requests
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from bs4 import BeautifulSoup
import time

In [26]:
imis_df = pd.read_csv('data/data_rf1_forecast.csv')

In [27]:
imis_df

,Unnamed: 0,datum,station_code,sector_id,warnreg,elevation_station,forecast_initial_date,forecast_end_date,dangerLevel,elevation_th,...,ssi_pwl,sk38_pwl,sn38_pwl,ccl_pwl,ssi_pwl_100,sk38_pwl_100,sn38_pwl_100,ccl_pwl_100,Pen_depth,min_ccl_pen
0,0,1997-11-11,KES2,7113.0,15.0,2700.0,1997-11-11 17:00:00,1997-11-12 17:00:00,1.0,2000.0,...,2.02,1.02,1.92,0.30,2.02,1.02,1.92,0.30,44.028391,0.17
1,1,1997-11-11,SIM2,6113.0,15.0,2400.0,1997-11-11 17:00:00,1997-11-12 17:00:00,2.0,2000.0,...,6.00,6.00,6.00,4.00,6.00,6.00,6.00,4.00,37.271809,0.26
2,2,1997-11-11,DTR2,6113.0,15.0,2100.0,1997-11-11 17:00:00,1997-11-12 17:00:00,2.0,2000.0,...,1.44,0.44,1.37,0.12,1.44,0.44,1.37,0.12,38.369101,0.12
3,3,1997-11-11,MEI2,2221.0,15.0,2200.0,1997-11-11 17:00:00,1997-11-12 17:00:00,1.0,2000.0,...,6.00,6.00,6.00,0.20,6.00,6.00,6.00,0.20,20.400000,4.00
4,4,1997-11-11,SPN2,4232.0,15.0,2600.0,1997-11-11 17:00:00,1997-11-12 17:00:00,2.0,2000.0,...,6.00,6.00,6.00,4.00,6.00,6.00,6.00,4.00,42.332551,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292832,292832,2020-05-04,FIR2,1242.0,21.0,2100.0,2020-05-04 17:00:00,2020-05-05 17:00:00,2.0,1800.0,...,6.00,6.00,6.00,4.00,6.00,6.00,6.00,4.00,6.299643,3.00
292833,292833,2020-05-04,GRA2,1311.0,21.0,2000.0,2020-05-04 17:00:00,2020-05-05 17:00:00,2.0,1800.0,...,6.00,6.00,6.00,4.00,6.00,6.00,6.00,4.00,6.881834,3.00
292834,292834,2020-05-04,SHE2,1213.0,21.0,1900.0,2020-05-04 17:00:00,2020-05-05 17:00:00,2.0,1800.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
292835,292835,2020-05-04,ELS2,1231.0,21.0,2100.0,2020-05-04 17:00:00,2020-05-05 17:00:00,2.0,1800.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Remove Unnecessary Column
imis_df.drop(columns=['Unnamed: 0'], inplace=True)

As we can see, the coordinates of the IMIS weather stations are missing in this dataset. I add them from another dataset containing the coordinates of all IMIS stations in Switzerland.

In [29]:
stations_csv = pd.read_csv('data/stations.csv')
imis_stations = imis_df['station_code'].unique()

# Check which stations are in the CSV
csv_stations = stations_csv['station_code'].unique()
found_in_csv = [s for s in imis_stations if s in csv_stations]
not_in_csv = [s for s in imis_stations if s not in csv_stations]

# Merging 
imis_df = imis_df.merge(stations_csv[['station_code', 'lon', 'lat']], 
                         on='station_code', 
                         how='left')

if imis_df['lon'].isna().sum() > 0:
    still_missing = imis_df[imis_df['lon'].isna()]['station_code'].unique()
    print(f"\nStations STILL missing coordinates: {sorted(still_missing)}")

In [30]:
imis_df

,datum,station_code,sector_id,warnreg,elevation_station,forecast_initial_date,forecast_end_date,dangerLevel,elevation_th,set,...,sn38_pwl,ccl_pwl,ssi_pwl_100,sk38_pwl_100,sn38_pwl_100,ccl_pwl_100,Pen_depth,min_ccl_pen,lon,lat
0,1997-11-11,KES2,7113.0,15.0,2700.0,1997-11-11 17:00:00,1997-11-12 17:00:00,1.0,2000.0,train,...,1.92,0.30,2.02,1.02,1.92,0.30,44.028391,0.17,9.898139,46.621284
1,1997-11-11,SIM2,6113.0,15.0,2400.0,1997-11-11 17:00:00,1997-11-12 17:00:00,2.0,2000.0,train,...,6.00,4.00,6.00,6.00,6.00,4.00,37.271809,0.26,8.980823,46.467455
2,1997-11-11,DTR2,6113.0,15.0,2100.0,1997-11-11 17:00:00,1997-11-12 17:00:00,2.0,2000.0,train,...,1.37,0.12,1.44,0.44,1.37,0.12,38.369101,0.12,8.869060,46.542866
3,1997-11-11,MEI2,2221.0,15.0,2200.0,1997-11-11 17:00:00,1997-11-12 17:00:00,1.0,2000.0,train,...,6.00,0.20,6.00,6.00,6.00,0.20,20.400000,4.00,8.551014,46.743768
4,1997-11-11,SPN2,4232.0,15.0,2600.0,1997-11-11 17:00:00,1997-11-12 17:00:00,2.0,2000.0,train,...,6.00,4.00,6.00,6.00,6.00,4.00,42.332551,0.16,8.117624,46.229444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292832,2020-05-04,FIR2,1242.0,21.0,2100.0,2020-05-04 17:00:00,2020-05-05 17:00:00,2.0,1800.0,test,...,6.00,4.00,6.00,6.00,6.00,4.00,6.299643,3.00,8.064403,46.668777
292833,2020-05-04,GRA2,1311.0,21.0,2000.0,2020-05-04 17:00:00,2020-05-05 17:00:00,2.0,1800.0,test,...,6.00,4.00,6.00,6.00,6.00,4.00,6.881834,3.00,6.783550,46.346062
292834,2020-05-04,SHE2,1213.0,21.0,1900.0,2020-05-04 17:00:00,2020-05-05 17:00:00,2.0,1800.0,test,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.812449,46.748825
292835,2020-05-04,ELS2,1231.0,21.0,2100.0,2020-05-04 17:00:00,2020-05-05 17:00:00,2.0,1800.0,test,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.641641,46.529602


In [31]:
imis_df.columns

Index(['datum', 'station_code', 'sector_id', 'warnreg', 'elevation_station',
       'forecast_initial_date', 'forecast_end_date', 'dangerLevel',
       'elevation_th', 'set', 'Qs', 'Ql', 'TSG', 'Qg0', 'Qr', 'OLWR', 'ILWR',
       'LWR_net', 'OSWR', 'ISWR', 'Qw', 'pAlbedo', 'ISWR_h', 'ISWR_diff',
       'ISWR_dir', 'TA', 'TSS_mod', 'TSS_meas', 'T_bottom', 'RH', 'VW',
       'VW_drift', 'DW', 'MS_Snow', 'HS_mod', 'HS_meas', 'hoar_size',
       'wind_trans24', 'wind_trans24_7d', 'wind_trans24_3d', 'HN24', 'HN72_24',
       'HN24_7d', 'SWE', 'MS_water', 'MS_Wind', 'MS_Rain', 'MS_SN_Runoff',
       'MS_Sublimation', 'MS_Evap', 'TS0', 'TS1', 'TS2', 'Sclass2', 'zSd_mean',
       'Sd', 'zSn', 'Sn', 'zSs', 'Ss', 'zS4', 'S4', 'zS5', 'S5', 'pwl_100',
       'pwl_100_15', 'base_pwl', 'ssi_pwl', 'sk38_pwl', 'sn38_pwl', 'ccl_pwl',
       'ssi_pwl_100', 'sk38_pwl_100', 'sn38_pwl_100', 'ccl_pwl_100',
       'Pen_depth', 'min_ccl_pen', 'lon', 'lat'],
      dtype='object')

**Feature Table**

| Index | Variable Name         | Description                                                          |
| :---- | :-------------------- | :------------------------------------------------------------------- |
| 0     | `datum`                 | Date                                                                 |
| 1     | `station_code`          | Station Identifier                                                   |
| 2     | `sector_id`             | Sector ID                                                            |
| 3     | `warnreg`               | Warning Region ID                                                    |
| 4     | `elevation_station`     | Elevation of the station                                             |
| 5     | `forecast_initial_date` | Start date of the forecast validity                                  |
| 6     | `forecast_end_date`     | End date of the forecast validity                                    |
| 7     | `dangerLevel`           | Target variable: Forecast danger level                               |
| 8     | `elevation_th`          | Elevation threshold from the forecast                                |
| 9     | `set`                   | Data split identifier (e.g., train/test)                             |
| 10    | `Qs`                    | Sensible heat [$W m^{-2}$]                                            |
| 11    | `Ql`                    | Latent heat [$W m^{-2}$]                                              |
| 12    | `TSG`                   | Ground temperature [$^{\circ}C$]                                      |
| 13    | `Qg0`                   | Ground heat at soil interface [$W m^{-2}$]                            |
| 14    | `Qr`               | Rain energy [$W m^{-2}$]                                              |
| 15    | `OLWR`                  | Outgoing long-wave radiation [$W m^{-2}$]                             |
| 16    | `ILWR`                  | Incoming long-wave radiation [$W m^{-2}$]                             |
| 17    | `LWR_net`               | Net long-wave radiation [$W m^{-2}$]                                 |
| 18    | `OSWR`                  | Reflected short-wave radiation [$W m^{-2}$]                          |
| 19    | `ISWR`                  | Incoming short-wave radiation [$W m^{-2}$]                            |
| 20    | `Qw`                    | Net short-wave radiation [$W m^{-2}$]                                |
| 21    | `pAlbedo`               | Parametrized albedo [-]                                              |
| 22    | `ISWR_h`                | Incoming short wave on the horizontal [$W m^{-2}$]                    |
| 23    | `ISWR_diff`             | Diffuse incoming short wave [$W m^{-2}$]                              |
| 24    | `ISWR_dir`              | Direct incoming short wave [$W m^{-2}$]                               |
| 25    | `TA`                    | Air temperature [$^{\circ}C$]                                        |
| 26    | `TSS_mod`               | Surface temperature [$^{\circ}C$] (Modelled)                          |
| 27    | `TSS_meas`              | Surface temperature [$^{\circ}C$] (Measured)                          |
| 28    | `T_bottom`              | Bottom temperature [$^{\circ}C$]                                     |
| 29    | `RH`                    | Relative humidity [-]                                                |
| 30    | `VW`                    | Wind velocity [$m s^{-1}$]                                           |
| 31    | `VW_drift`              | Wind velocity drift [$m s^{-1}$]                                     |
| 32    | `DW`                    | Wind direction [$^{\circ}$]                                          |
| 33    | `MS_Snow`               | Solid precipitation rate [$kg s^{-2} h^{-1}$]                         |
| 34    | `HS_mod`                | Snow height [cm] (Modelled)                                          |
| 35    | `HS_meas`               | Snow height [cm] (Measured)                                          |
| 36    | `hoar_size`             | Hoar size [cm]                                                       |
| 37    | `wind_trans24`          | 24 h wind drift [cm]                                                 |
| 38    | `wind_trans24_7d`       | 7 d wind drift [cm]                                                  |
| 39    | `wind_trans24_3d`       | 3 d wind drift [cm]                                                  |
| 40    | `HN24`                  | 24 h height of new snow [cm]                                         |
| 41    | `HN72_24`               | 3 d sum of daily height of new snow [cm]                             |
| 42    | `HN24_7d`               | 7 d sum of daily height of new snow [cm]                             |
| 43    | `SWE`                   | Snow water equivalent [$kg m^{-2}$]                                  |
| 44    | `MS_water`              | Total amount of water [$kg m^{-2}$]                                  |
| 45    | `MS_Wind`               | Erosion mass loss [$kg m^{-2}$]                                      |
| 46    | `MS_Rain`               | Rain rate [$kg s^{-2} h^{-1}$]                                       |
| 47    | `MS_SN_Runoff`          | Virtual lysimeter [$kg s^{-2} h^{-1}$]                               |
| 48    | `MS_Sublimation`        | Sublimation mass [$kg m^{-2}$]                                       |
| 49    | `MS_Evap`               | Evaporated mass [$kg m^{-2}$]                                        |
| 50    | `TS0`                   | Snow temperature at 0.25 m [$^{\circ}C$]                              |
| 51    | `TS1`                   | Snow temperature at 0.5 m [$^{\circ}C$]                               |
| 52    | `TS2`                   | Snow temperature at 1 m [$^{\circ}C$]                                |
| 53    | `Sclass2`               | Stability class [-]                                                  |
| 54    | `zSd_mean`              | Mean depth of deformation rate stability index [cm]                  |
| 55    | `Sd`                    | Deformation rate stability index [-]                                 |
| 56    | `zSn`                   | Depth of natural stability index [cm]                                |
| 57    | `Sn`                    | Natural stability index [-]                                          |
| 58    | `zSs`                   | Depth of Sk38 skier stability index [m]                              |
| 59    | `Ss`                    | Sk38 skier stability index [-]                                       |
| 60    | `zS4`                   | Depth of structural stability index [cm]                             |
| 61    | `S4`                    | Structural stability index [-]                                       |
| 62    | `zS5`                   | Depth of stability index 5 [cm]                                      |
| 63    | `S5`                    | Stability index 5 [-]                                                |
| 64    | `pwl_100`               | Persistent weak layer(s) in the 100 cm from the surface [-]            |
| 65    | `pwl_100_15`            | Persistent weak layer(s) at depths between 15 and 100 cm [-]         |
| 66    | `base_pwl`              | Persistent weak layer at bottom [-]                                  |
| 67    | `ssi_pwl`               | Structural stability index at weak layer [-]                         |
| 68    | `sk38_pwl`              | Sk38 skier stability index at weak layer [-]                         |
| 69    | `sn38_pwl`              | Natural stability index at weak layer [-]                            |
| 70    | `ccl_pwl`               | Critical cut length at weak layer [m]                                |
| 71    | `ssi_pwl_100`           | Structural stability index at surface weak layer [-]                 |
| 72    | `sk38_pwl_100`          | Sk38 skier stability index at surface weak layer [-]                 |
| 73    | `sn38_pwl_100`          | Natural stability index at surface weak layer [-]                    |
| 74    | `ccl_pwl_100`           | Critical cut length at surface weak layer [m]                        |
| 75    | `Pen_depth`             | Skier penetration depth [cm]                                         |
| 76    | `min_ccl_pen`           | Min critical cut length at a deeper layer of the penetration depth [m] |
| 77    | `lon`                   | Longitude of the station                                            |
| 78    | `lat`                   | Latitude of the station                                             |

#### **1.1 Data Cleaning & Imputation**

I start by checking for missing values in the dataset to understand the extent of data cleaning required.

In [32]:
# Missing Values
missing_values = pd.DataFrame(imis_df.isnull().sum(), columns=['Missing_Count'])
missing_values_df = missing_values.sort_values(by='Missing_Count', ascending=False)
missing_values_df['Missing_Percentage'] = (missing_values_df['Missing_Count'] / len(imis_df)) * 100
missing_values_df.head(25)

,Missing_Count,Missing_Percentage
elevation_th,33692,11.505377
TS2,30313,10.351492
TS1,28884,9.863508
TS0,25859,8.830510
Sd,15449,5.275631
Sn,15449,5.275631
S4,14185,4.843992
Ss,14185,4.843992
Pen_depth,11315,3.863924
min_ccl_pen,10986,3.751575


The missing value table reveals that several key features have significant missing data, particularly the snow temperature measurements (TS0, TS1, TS2) and stability indices (Sd, Sn, Ss). Let's dig a bit deeper to see if the missingness is related to specific stations or danger levels and whether there are correlations between missing values in different features.

In [ ]:
# Check if missingness is related to specific conditions
print("Missing values by station:")
print(imis_df[imis_df['TS0'].isnull()].groupby('station_code').size().sort_values(ascending=False).head())

print("\nMissing values of elevation_th by danger level:")
print(imis_df[imis_df['elevation_th'].isnull()].groupby('dangerLevel').size())

# Check correlation between missing values
missing_matrix = imis_df[['TS0', 'TS1', 'TS2', 'Sd', 'Sn', 'Ss', 'elevation_th']].isnull()
print("\nCorrelation of missingness:")
print(missing_matrix.corr())

Missing values by station:
station_code
FOU2    2996
BOR2    2910
DAV4    2301
SIM2    2057
DAV5    2027
dtype: int64

Missing values by danger level:
dangerLevel
1.0    32539
2.0      459
3.0      533
4.0      161
dtype: int64

Correlation of missingness:
                   TS0       TS1       TS2        Sd        Sn        Ss  \
TS0           1.000000  0.924263  0.892056  0.020123  0.020123  0.015270   
TS1           0.924263  1.000000  0.954588  0.030595  0.030595  0.021384   
TS2           0.892056  0.954588  1.000000  0.027871  0.027871  0.019300   
Sd            0.020123  0.030595  0.027871  1.000000  1.000000  0.573174   
Sn            0.020123  0.030595  0.027871  1.000000  1.000000  0.573174   
Ss            0.015270  0.021384  0.019300  0.573174  0.573174  1.000000   
elevation_th  0.060342  0.050528  0.048564  0.111476  0.111476  0.067243   

              elevation_th  
TS0               0.060342  
TS1               0.050528  
TS2               0.048564  
Sd                

The analysis above suggests the following:
- `elevation_th`: missingness is related to danger levels. SLF doesn't publish the elevation for danger level 1. We impute with 0 for danger level 1, and with the station elevation for danger levels 2-5.
- `TS0`, `TS1`, `TS2`: The high correlation in their missingness (0.89 - 0.95) confirms they fail together. This is because these sensors require a minimum snow depth (often >100 cm) to provide valid temperatur readings. 
- `Sd`, `Sn`, `Ss`: These stability indices are also highly correlated in their missingness (0.57 - 1.00). In SNOWPACK simulations, these values are often undefined if the model cannot find a distinct "persistent weak layer" (PWL). I assign the maximum observed value for these indices to indicate the absence of a weak layer.
- 

**Next Steps**
1. Missing Values
2. Class Imbalance
3. Correlation Analysis
4. Elevation Filtering 